# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [30]:
def square_root_generator(limit):
    n = 1
    sum_output = 0
    while n <= limit:
      sum_output = sum_output + n ** 0.5
      yield n ** 0.5, sum_output
      n += 1



# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)



(1.0, 1.0)
(1.4142135623730951, 2.414213562373095)
(1.7320508075688772, 4.146264369941973)
(2.0, 6.146264369941973)
(2.23606797749979, 8.382332347441762)


In [31]:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

(1.0, 1.0)
(1.4142135623730951, 2.414213562373095)
(1.7320508075688772, 4.146264369941973)
(2.0, 6.146264369941973)
(2.23606797749979, 8.382332347441762)
(2.449489742783178, 10.83182209022494)
(2.6457513110645907, 13.47757340128953)
(2.8284271247461903, 16.30600052603572)
(3.0, 19.30600052603572)
(3.1622776601683795, 22.4682781862041)
(3.3166247903554, 25.7849029765595)
(3.4641016151377544, 29.249004591697254)
(3.605551275463989, 32.854555867161245)


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [32]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [33]:
%%capture
!pip install "dlt[duckdb]" # Install dlt with all the necessary DuckDB dependencies

In [34]:
import dlt

# define the connection to load to.
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators_1')

In [35]:
#load people_1 generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.27 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators_1
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707828811.8721411 is LOADED and contains no failed jobs


In [36]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

people = conn.sql("SELECT * FROM people").df()
display(people)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707828811.8721411,pjbII0VWzBuDAg
1,2,Person_2,27,City_A,1707828811.8721411,6BbaVQBNkdufBA
2,3,Person_3,28,City_A,1707828811.8721411,WfOASzaviDvfZg
3,4,Person_4,29,City_A,1707828811.8721411,qHAOVxqVTg1lJA
4,5,Person_5,30,City_A,1707828811.8721411,HCqETUcIo6egig


In [37]:
#Append people_2 generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.35 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators_1
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707828842.4869087 is LOADED and contains no failed jobs


In [38]:
# show outcome
# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

people = conn.sql("SELECT * FROM people").df()
display(people)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707828811.8721411,pjbII0VWzBuDAg,None
1,2,Person_2,27,City_A,1707828811.8721411,6BbaVQBNkdufBA,None
2,3,Person_3,28,City_A,1707828811.8721411,WfOASzaviDvfZg,None
3,4,Person_4,29,City_A,1707828811.8721411,qHAOVxqVTg1lJA,None
4,5,Person_5,30,City_A,1707828811.8721411,HCqETUcIo6egig,None
5,3,Person_3,33,City_B,1707828842.4869087,d/8alrYbW8u0qA,Job_3
6,4,Person_4,34,City_B,1707828842.4869087,JLZUln8vYdVhbQ,Job_4
7,5,Person_5,35,City_B,1707828842.4869087,bc1Bu7Wggwf30g,Job_5
8,6,Person_6,36,City_B,1707828842.4869087,y00c4G/bmyyuCQ,Job_6
9,7,Person_7,37,City_B,1707828842.4869087,R5CeDn5PqCK9NQ,Job_7


In [39]:
people['age'].sum()

353

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [40]:
info = generators_pipeline.run(people_1(),
										table_name="people_new",
										write_disposition="replace",
                    primary_key="id")
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators_1
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707828935.535401 is LOADED and contains no failed jobs


In [41]:
info = generators_pipeline.run(people_2(),
										table_name="people_new",
										write_disposition="merge",
                    primary_key="id")
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.45 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators_1
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707828938.553723 is LOADED and contains no failed jobs


In [42]:
# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n  people new table below:")



Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_new          │
└─────────────────────┘




  people new table below:


In [43]:
people_new = conn.sql("SELECT * FROM people_new").df()
display(people_new)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707828935.535401,amJsS1VsUVxOUw,None
1,2,Person_2,27,City_A,1707828935.535401,R9pBFSFUuFPyVQ,None
2,5,Person_5,35,City_B,1707828938.553723,Ilnqt7FaQILcag,Job_5
3,8,Person_8,38,City_B,1707828938.553723,ViBkqq89OOU3dw,Job_8
4,7,Person_7,37,City_B,1707828938.553723,r6Lamrj1Fo47vA,Job_7
5,4,Person_4,34,City_B,1707828938.553723,ArfZtWKuytrrXw,Job_4
6,3,Person_3,33,City_B,1707828938.553723,mfMd6/oaA6Zshg,Job_3
7,6,Person_6,36,City_B,1707828938.553723,VX6OX7JTGIqYJQ,Job_6


In [44]:
people_new['age'].sum()

266

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX